In [ ]:
# Importa as bibliotecas
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

try:
    import yfinance as yf
    yf.pdr_override()
except:
    !pip install yfinance
try:
    from matplotlib import pyplot as plt
except:    
    !pip install -U matplotlib

try:
    import six
    import sys
    sys.modules['sklearn.externals.six'] = six
    import mlrose
except:
    !pip install mlrose

    

# Otimização

In [ ]:
# Busca os preços das ações
## Define as ações
assets = ['PETR4.SA','VALE3.SA','ITUB4.SA','ABEV3.SA','BBDC4.SA','BBAS3.SA','WEGE3.SA','BPAC11.SA','SANB11.SA','ITSA4.SA']

## Define as datas  ---- PRIMEIRO PERÍODO
start = '2022-01-01'
end = '2022-12-31'

# Busca os preços ajustados
prices = yf.download(assets, start = start, end = end)['Adj Close']

In [ ]:
# Plata o Histórico de Preços

px.line(prices, width=1100)

In [ ]:
# Calcula os retornos e retira dados faltantes
returns = prices.pct_change().dropna()
returns

In [ ]:
# Plotando a matriz de correlação em um mapa de calor:

px.imshow(returns.corr(), title = 'Matriz de Correlação', text_auto=True, height=900, width=850 , color_continuous_scale='viridis')

In [ ]:
# Simulação de Monte Carlo

def generate_wallets(df, num_portfolios = 100000, risk_free = 0.1325):
    # vetores de dados
    portfolio_weights = []
    portfolio_exp_returns = []
    portfolio_vol = []
    portfolio_sharpe = []

    # retorno simples 
    r = df.pct_change()
    mean_returns = r.mean() * 252

    # matriz de covariância 
    covariance = np.cov(r[1:].T)

    for i in range(num_portfolios):
        # gerando pesos aleatórios
        k = np.random.rand(len(df.columns))
        w = k / sum (k)

        # retorno
        R = np.dot(mean_returns, w)

        # risco
        vol = np.sqrt(np.dot(w.T, np.dot(covariance, w))) * np.sqrt(252)

        # sharpe ratio
        sharpe = (R - risk_free)/vol

        portfolio_weights.append(w)
        portfolio_exp_returns.append(R)
        portfolio_vol.append(vol)
        portfolio_sharpe.append(sharpe)

    wallets = {'weights': portfolio_weights,
              'returns': portfolio_exp_returns,
              'vol':portfolio_vol,
              'sharpe': portfolio_sharpe}

    return wallets

In [ ]:
# Melhor carteira dentre as simulações

def best_portfolio(wallets):
    sharpe = wallets['sharpe']
    weights = wallets['weights']
    
    indice = np.array(sharpe).argmax()
    return weights[indice], sharpe[indice]

In [ ]:
# Executando as simulações e encontrando a Carteira Otimizada Randomicamente

wallets = generate_wallets(prices)
best_portfolio(wallets)

In [ ]:
# Salvando Carteira Otimizada Randomicamente

pesos = pd.DataFrame()
pesos['assets'] = prices.columns
pesos['weights'] = best_portfolio(wallets)[0]
pesos.to_csv('pesos.csv')

In [ ]:
# Plota a Carteira Otimizada Randomicamente

px.pie(pesos, title = 'Composição da Carteira Otimizada Randomicamente', names= 'assets',values= 'weights', width=600, color_discrete_sequence=px.colors.sequential.Viridis_r)

In [ ]:
# Plota a fronteira eficiente de ativos

def plot_efficient_frontier(wallets):
    vol = wallets['vol']
    returns = wallets['returns']
    sharpe = wallets['sharpe']

    indice = np.array(sharpe).argmax()
    y_axis = returns[indice]
    X_axis = vol[indice]

    grafico = px.scatter(wallets,x='vol', y='returns', color='sharpe', color_continuous_scale='viridis', height=650, width=1220)
    grafico.update_layout(xaxis_title = 'Volatilidade', yaxis_title = 'Retorno Experado')
    grafico.show()

In [ ]:
# Chamada da função de plotagem com as carteiras geradas randomicamente

plot_efficient_frontier(wallets)

## Algoritimos de Otimização

In [ ]:
#Função fitness, requisito da otimização via mlrose

def fitness_function(solucao):
    # retorno simples 
    r = returns
    mean_returns = r.mean() * 252

    # matriz de covariância 
    covariance = np.cov(r[1:].T)

    # gerando pesos aleatórios
    k = solucao
    w = k / sum (k)

    # retorno
    R = np.dot(mean_returns, w)

    # risco
    vol = np.sqrt(np.dot(w.T, np.dot(covariance, w))) * np.sqrt(252)
  
    # sharpe ratio

    risk_free = 0.1325
    sharpe = (R - risk_free)/vol

    return sharpe

fitness = mlrose.CustomFitness(fitness_function)

In [ ]:
# Problema de maximização

problema_maximizacao = mlrose.ContinuousOpt(length=10, fitness_fn=fitness,
                                            maximize = True, min_val = 0, max_val = 1)

In [ ]:
# Passa os reultados dos pesos para percentual

def alocation_normalize(solution):
  solution = solution / solution.sum()
  return solution

## Hill Climb

In [ ]:
# Obtém a Carteira Otimizada pelo algoritmo Hill Climb

hill_climb_weights, hill_climb_sharpe = mlrose.hill_climb(problema_maximizacao, random_state = 1)
hill_climb_weights = alocation_normalize(hill_climb_weights)
hill_climb_weights, hill_climb_sharpe

In [ ]:
# Salva Carteira Otimizada pelo algoritmo Hill Climb

pesos_hill_climb = pd.DataFrame()
pesos_hill_climb['assets'] = prices.columns
pesos_hill_climb['weights'] = hill_climb_weights
pesos_hill_climb.to_csv('pesos_hill_climb.csv')
pesos_hill_climb

In [ ]:
# Plota a Carteira Otimizada pelo algoritmo Hill Climb

px.pie(pesos_hill_climb, title = 'Composição da Carteira Otimizada - Hill Climb', names= 'assets',values= 'weights', width=600, color_discrete_sequence=px.colors.sequential.Viridis_r)

## Simulated Annealing

In [ ]:
# Obtém a Carteira Otimizada pelo algoritmo Simulated Annealing

simulated_annealing_weights, simulated_annealing_sharpe = mlrose.simulated_annealing(problema_maximizacao, random_state=1)
simulated_annealing_weights = alocation_normalize(simulated_annealing_weights)
simulated_annealing_weights, simulated_annealing_sharpe

In [ ]:
# Salva a Carteira Otimizada pelo algoritmo Simulated Annealing

pesos_sa = pd.DataFrame()
pesos_sa['assets'] = prices.columns
pesos_sa['weights'] = simulated_annealing_weights
pesos_sa.to_csv('pesos_sa.csv')
pesos_sa

In [ ]:
# Plota a Carteira Otimizada pelo algoritmo Simulated Annealing

px.pie(pesos_sa, title = 'Composição da Carteira Otimizada - Simulated Annealing', names= 'assets',values= 'weights', width=600, color_discrete_sequence=px.colors.sequential.Viridis_r)

## Genetic Alg

In [ ]:
# Fitness function compatível com o Genetic Alg

def fitness_function_ag(solucao):
    # retorno simples 
    r = returns
    mean_returns = r.mean() * 252

    # matriz de covariância 
    covariance = np.cov(r[1:].T)

    # gerando pesos aleatórios
    k = solucao
    w = k / sum (k)

    # retorno
    R = np.dot(mean_returns, w)

    # risco
    vol = np.sqrt(np.dot(w.T, np.dot(covariance, w))) * np.sqrt(252)
  
    # sharpe ratio

    risk_free = 0.1325
    sharpe = (R - risk_free)/vol

    if sharpe > 0:
        return sharpe
    else:
        return 0


fitness_ag = mlrose.CustomFitness(fitness_function_ag)

problema_maximizacao_ag = mlrose.ContinuousOpt(length=10, fitness_fn=fitness_ag,
                                            maximize = True, min_val = 0, max_val = 1)


In [ ]:
# Obtém a Carteira Otimizada pelo Algoritmo Genético

genetic_alg_weights, genetic_alg_sharpe =  mlrose.genetic_alg(problema_maximizacao_ag, random_state = 1)
genetic_alg_weights = alocation_normalize(genetic_alg_weights)
genetic_alg_weights, genetic_alg_sharpe

In [ ]:
# Salva a Carteira Otimizada pelo Algoritmo Genético

pesos_ga = pd.DataFrame()
pesos_ga['assets'] = prices.columns
pesos_ga['weights'] = genetic_alg_weights
pesos_ga.to_csv('pesos_ga.csv')
pesos_ga

In [ ]:
# Plota a Carteira Otimizada pelo Algoritmo Genético

px.pie(pesos_ga, title = 'Composição da Carteira Otimizada - Algorítmo Genético', names= 'assets',values= 'weights', width=600, color_discrete_sequence=px.colors.sequential.Viridis_r)

# Verificação de Desempenho

In [ ]:
# Busca os preços das ações
## Define as ações
assets = ['PETR4.SA','VALE3.SA','ITUB4.SA','ABEV3.SA','BBDC4.SA','BBAS3.SA','WEGE3.SA','BPAC11.SA','SANB11.SA','ITSA4.SA']

## Define as datas  ---- SEGUNDO PERÍODO  
start = '2023-01-01'
end = '2023-12-31'

# Busca os preços ajustados
prices = yf.download(assets, start = start, end = end)['Adj Close']
prices

In [ ]:
# Calcula os retornos e retira dados faltantes
returns = prices.pct_change().dropna()
returns

In [ ]:
def calculaEvolucao (carteira, initial_money=10000):
    pesos = carteira['weights'].array
    initial_money = initial_money

    qtd = (pesos * initial_money / prices.head(1)).values

    evolucao = qtd * prices
    evolucao['Carteira'] = evolucao.sum(axis=1)

    final_money = evolucao['Carteira'].tail(1).values.sum()

    return evolucao['Carteira']

In [ ]:
# Carregando as carteiras otmizadas

carteira_randomica = calculaEvolucao (pd.read_csv('pesos.csv'))
carteira_hill_climb = calculaEvolucao (pd.read_csv('pesos_hill_climb.csv'))
carteira_simulated_annealing = calculaEvolucao (pd.read_csv('pesos_sa.csv'))
carteira_genetic_alg = calculaEvolucao(pd.read_csv('pesos_ga.csv'))

In [ ]:
#Carteira Pesos Iguais

n_assets = len(returns.columns)

pesos_iguais = []
for i in returns.columns:
    pesos_iguais.append(1/n_assets)

pesos_iguais = pd.array(pesos_iguais)

initial_money = 10000

qtd2 = (pesos_iguais * initial_money / prices.head(1)).values

evolucao2 = qtd2 * prices
evolucao2['Carteira'] = evolucao2.sum(axis=1)
evolucao2

final_money2 = evolucao2['Carteira'].tail(1).values.sum()
final_money2

In [ ]:
#BOVA

prices_bova = yf.download('BOVA11.SA', start=start, end=end)['Adj Close']
qtd_bova = initial_money / prices_bova.head(1).values
evolucao_bova = qtd_bova*prices_bova
final_money_bova = evolucao_bova.tail(1).values.sum()

In [ ]:
grafico = pd.DataFrame()
grafico['Carteira Otimizada - Randômica'] = carteira_randomica
grafico['Carteira Otimizada - Hill Climb'] = carteira_hill_climb
grafico['Carteira Otimizada - Simulated Annealing'] = carteira_simulated_annealing
grafico['Carteira Otimizada - Genetic Alg'] = carteira_genetic_alg
grafico['Carteira Pesos Iguais'] = evolucao2['Carteira']
grafico['BOVA'] = evolucao_bova

In [ ]:
figura = px.line(title='Comparativo - Evolução do Patrimônio')
for i in grafico:
  figura.add_scatter(x = grafico.index, y = grafico[i], name = i)

figura.update_layout(height=750)
figura.update_layout(font_size=18)
figura.show()